In [ ]:
# generate random data matrix
n,d = 6,4
X = randn(n,d)

# optional: give it linearly dependent columns
X[:,3] = X[:,2]

In [ ]:
U,σ,V = svd(X)

In [ ]:
U'*U

In [ ]:
V'*V

In [ ]:
σ

In [ ]:
# decomposition is just as good if we ignore the 0 in sigma and reduce r by 1
norm(X - U[:,1:3]*diagm(σ[1:3])*(V[:,1:3])')

In [ ]:
# form data from noisy linear model
w♮ = randn(d)
y = X*w♮ + .1*randn(n);

In [ ]:
# solve least squares problem to estimate w
w = V*diagm(σ.^(-1))*U'*y
# w = V[:,1:3]*diagm(σ[1:3].^(-1))*(U[:,1:3])'*y

In [ ]:
# how good is our estimate of w?
norm(w - w♮)

In [ ]:
# compute mean square error
mean((y - X*w).^2)

In [ ]:
# let's use the shorthand
w_backslash = X \ y
norm(w_backslash - w)